In [1]:
import numpy as np
import random
import csv

In [2]:
num_patterns = 3
num_positive = 50
num_negative = 50

In [8]:
met = list(csv.reader(open('../data/metadata.csv')))
tempo = int(met[0][0])
bpm = int(met[1][0])
tpb = int(met[2][0])
t_ptc =int(met[3][0])
t_amp = int(met[4][0])
t_dur = float(met[5][0])

In [3]:
pat = []
lbl = []
for i in range (0, num_patterns):
 lbl.append(i+1)
 patfile = '../data/pat_' + str(i) + '.csv'
 pt = []
 with open(patfile) as csvfile:
  rdr = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC)
  for row in rdr:
   pt.append(row)
 pat.append(pt)

In [11]:
## function to create positive and negative data
## if returns a value within the delta boundaries of positive, or outside the delta boundaries if negative

#positive data
def rand_pos(val, delta, lower, upper, flag):
 if(flag == 0):
  res = 0
  if(val-delta > lower and val+delta < upper):
   res = random.randint(val-delta, val+delta)   
  elif(val-delta <= lower):
   res = random.randint(lower, val+delta)
  elif(val+delta >= upper):
   res = random.randint(val-delta, upper)   
 if(flag == 1):
  res = 0
  if(val-delta > lower and val+delta < upper):
   res = random.uniform(val-delta, val+delta)   
  elif(val-delta <= lower):
   res = random.uniform(lower, val+delta)   
  elif(val+delta >= upper):
   res = random.uniform(val-delta, upper)   
 res = round(res, 2)
 return res


#negative data
def rand_neg(val, delta, lower, upper, flag):
 if(flag == 0):  
  if(val-delta <= lower):
   res = random.randint(val+delta, upper)
  elif(val+delta >= upper):
   res = random.randint(lower, val-delta)
  else:
   rangechoices = ((lower,val-delta), (val+delta, upper))
   fromrange = random.choice(rangechoices)
   res = random.randint(*fromrange)   
 if(flag == 1):  
  if(val-delta <= lower):
   res = random.uniform(val+delta, upper)
  elif(val+delta >= upper):
   res = random.uniform(lower, val-delta)
  else:
   rangechoices = ((lower,val-delta), (val+delta, upper))
   fromrange = random.choice(rangechoices)
   res = random.uniform(*fromrange)   
 res = round(res, 2)
 return res

In [5]:
## functrions to create positive, negative and amplitude variants

#creating positive variants
def create_pos(patt, lbll, dst_pat, dst_lbl): 
 for j in range(0, num_positive):
  tmp = []
  for i in range (0, len(patt)):
   ptc_i = patt[i][0]
   amp_i = rand_pos(patt[i][1], t_amp, 0, 127, 0)
   dur_i = rand_pos(patt[i][2], t_dur, 0, 2.0, 1)

   tmp.append([ptc_i, amp_i, dur_i])

  dst_pat.append(tmp)  
  dst_lbl.append(lbll)
  
  
#create amplitude variations from +-1 to +-20
def ampl(patt, lbll, dst_pat, dst_lbl):
 for j in range(-20, 21):
  tmp = []
  for i in range (0, len(patt)):
   ptc_i = patt[i][0]
   amp_i = patt[i][1] - j
   dur_i = patt[i][2]
   
   tmp.append([ptc_i, amp_i, dur_i])

  dst_pat.append(tmp)  
  dst_lbl.append(lbll)
  
  
def create_neg(patt, lbll, dst_pat, dst_lbl):
 for j in range(0, num_negative):
  tmp = []
  for i in range (0, len(patt)):

   ptc_i = rand_neg(patt[i][0], t_ptc, 0, 127, 0)
   amp_i = rand_neg(patt[i][1], t_amp, 0, 127, 0)
   dur_i = rand_neg(patt[i][2], t_dur, 0, 2.0, 1)

   tmp.append([ptc_i, amp_i, dur_i])

  dst_pat.append(tmp)  
  dst_lbl.append(0)

In [14]:
## function to write training data to file

def write_data(in_pat, in_lbl, data_name, labels_name):
 with open(data_name, mode='w', newline='') as f:
  data_writer = csv.writer(f)

  for i in range(0, len(in_pat)):
   tt = []
   for j in range(0, len(in_pat[i])):
    for k in range(0, len(in_pat[i][j])):
     tt.append(in_pat[i][j][k])
   data_writer.writerow(tt)

 with open(labels_name, mode='w', newline='') as f:
  data_writer = csv.writer(f)

  for i in range(0, len(in_lbl)):
   tt = []
   tt.append(in_lbl[i])
   data_writer.writerow(tt)